## SVM 

In [12]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import glob;
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm
from sklearn.svm import SVC,SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from random import randrange
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedKFold

In [24]:
dir = os.getcwd()
paths = glob.glob(dir+'raw data/*PM20130101_20151231.csv')
datasets = {}
for path in paths:
    city_name = path.split('/',6)[6][:-36]
    
    print(city_name)
    with open(path, 'r') as fin:
        datasets[city_name] = pd.read_csv(path, index_col = 0)

{}


In [14]:
#Standadization
X = {}
y1 = {}
y = {}
standardized_X = {}
standardized_y = {}
for city in datasets:
    X[city] = datasets[city][datasets[city].columns[:-2]]
    y1[city] = datasets[city]['PM_Mean']
    y[city] = datasets[city]['PM_Class']
#print(X['shenyang'].columns)
#standardized_X = preprocessing.scale(X)
#dataframe2 = pd.DataFrame(standardized_X,columns = cancer['feature_names'])
#standardized_X = dataframe2[dataframe2.columns[:]]
for city in datasets:
    standardized_X[city] = preprocessing.scale(X[city])
    df = pd.DataFrame(standardized_X[city],columns = X[city].columns)
    standardized_X[city] = df[df.columns[:]]
#print(standardized_X['beijing'])

In [15]:
C = 1000.0 # SVM regularization parameter
length = 10000 
models = {}
scores = {}
for city in standardized_X:
    models[city] = (svm.LinearSVC(C=C),
              svm.SVC(kernel='rbf', gamma=0.7, C=C),
              svm.SVC(kernel='poly', degree=3, C=C))
    models[city] = (clf.fit(standardized_X[city][0:length], y[city][0:length]) for clf in models[city])
    scores[city] = (clf.score(standardized_X[city][0:length],y[city][0:length])for clf in models[city])
    print(city)
    print("The accuracy of different kernel SVM are:")
    for s in scores[city]:
        print(s)

In [16]:
# Select the best two features
model = ExtraTreesClassifier()
model.fit(standardized_X['beijing'], y['beijing'])
# display the relative importance of each attribute
print(model.feature_importances_)
importance = model.feature_importances_
X_new = standardized_X['beijing'][standardized_X['beijing'].columns.values[np.argsort(importance)[::-1][:2]]]
best_feature = X_new.columns[0]
best_feature2 = X_new.columns[1]
print("The most importance feature in this dataset are: ", best_feature," and ", best_feature2 )

KeyError: 'beijing'

In [10]:
# Randomly select 20 % data for testing and the rest for training
X_train = {}
X_test = {}
y_train = {}
y_test = {}
for city in standardized_X:
    X_train[city], X_test[city], y_train[city], y_test[city] = train_test_split(standardized_X[city], y[city], test_size = 0.2,random_state=0)

In [ ]:
''''#using random search, find the optimum hyperparameter values
model = SVC(gamma=0.7)
C_range = np.logspace(-1, 10, 6)
gamma_range = np.logspace(-9, 3, 6)
# specify parameters and distributions to sample from
param_dist = {"gamma": gamma_range,
              "C": C_range}
grid = GridSearchCV(estimator=model, param_grid=param_dist)
# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=n_iter_search)

random_search.fit(X_train['beijing'], y_train['beijing'])

# summarize the results of the random search
print(random_search.best_score_)
gamma_best = random_search.best_estimator_.gamma
print("The optimal value of gamma is :", gamma_best)
C_best = random_search.best_estimator_.C
print("The optimal value of C is :", C_best)''''

In [9]:
model_Gaussian = {}
predicted = {}
pred = {}
accuracy = {}
for city in X_train:
    model_Gaussian[city] = svm.SVC(kernel='rbf', gamma=0.7, C = 1000.0)
    model_Gaussian[city].fit(X_train[city],y_train[city])
    predicted[city] = model_Gaussian[city].predict(X_test[city])
    accuracy[city] = accuracy_score(y_test[city],predicted[city])
    pred[city] = cross_val_predict(model_Gaussian[city], X_test[city], y_test[city], cv=5)
    print(city)
    print("The accuracy is:", accuracy[city])
    print(metrics.classification_report(y_test[city], pred[city]))

beijing
The accuracy is: 0.739312877
             precision    recall  f1-score   support

          1       0.73      0.79      0.76      1516
          2       0.50      0.51      0.50      1023
          3       0.42      0.39      0.41       599
          4       0.32      0.24      0.27       300
          5       0.60      0.57      0.58       375

avg / total       0.58      0.59      0.58      3813

chengdu
The accuracy is: 0.732621589561
             precision    recall  f1-score   support

          1       0.65      0.70      0.68      1608
          2       0.56      0.56      0.56      1522
          3       0.46      0.43      0.44       576
          4       0.44      0.35      0.39       293
          5       0.66      0.55      0.60       216

avg / total       0.58      0.58      0.58      4215

guangzhou
The accuracy is: 0.812453300125
             precision    recall  f1-score   support

          1       0.82      0.85      0.83      2561
          2       0.60    

In [11]:
X_trn_bj = {}
X_test_bj = {}
y_trn_bj = {}
y_test_bj = {}
for i in range(3):
    X_trn_bj[i], X_test_bj[i], y_trn_bj[i], y_test_bj[i] = train_test_split(standardized_X['beijing'], y['beijing'], test_size = 0.3, random_state=0)

In [12]:
def finddifference(A,B):
    index = B.index.tolist()
    #print(index)
    #print("----------------------------------------------------------")
    C = B.as_matrix()
    num = []
    for i in range(len(A)):
        if C[i] != A[i]:
            num.append(index[i])
    return num

In [14]:
#print(X_trn_bj[1])
model_Gauss = {}
predicts = {}
model_Gauss[0] = svm.SVC(kernel='rbf', gamma=0.7, C = 1000.0)
model_Gauss[1] = svm.LinearSVC(C=1000.0)
model_Gauss[2] = svm.SVC(kernel='poly', degree=3, C=C)
for i in X_trn_bj:
    model_Gauss[i].fit(X_trn_bj[i],y_trn_bj[i])
    predicts[i] = model_Gauss[i].predict(X_test_bj[i])
    #print("Prediction:", len(predicts[i]))
    #print(y_test_bj.size)
    #print(y_test_bj[i])
print("Prediction:", len(predicts[1]))
print(y_test_bj[1].size)


Prediction: 5719
5719


In [41]:
Difference = {}
for i in range(3):
    Difference[i] = finddifference(predicts[i],y_test_bj[i])
    print(Difference[i])
    print("--------------------------------------")
    print(len(Difference[i]))


[1449, 7105, 16781, 14705, 17069, 5734, 17950, 777, 10911, 10906, 5586, 1070, 7376, 13837, 2761, 1851, 3814, 3781, 15679, 1564, 19043, 12872, 13290, 13511, 6584, 11825, 8863, 11672, 12788, 17951, 12089, 17623, 160, 2307, 13833, 6085, 4962, 15095, 18303, 1373, 2555, 17933, 15054, 6040, 17276, 17666, 2068, 8418, 16883, 3493, 17264, 3633, 14676, 9162, 18573, 15369, 1528, 200, 18961, 14996, 3021, 4582, 17650, 4073, 6608, 10260, 12388, 14444, 17149, 17068, 5382, 7371, 2176, 7090, 11760, 9105, 3024, 6347, 990, 5252, 14870, 10771, 10853, 9928, 6695, 13222, 6941, 3140, 15513, 1905, 14647, 4478, 18861, 11639, 13659, 10772, 17110, 1972, 8002, 1979, 15601, 5751, 13229, 6060, 15850, 7619, 9385, 15570, 3798, 8363, 4557, 18572, 16053, 11894, 5189, 13459, 11532, 18223, 1326, 1405, 16594, 867, 9083, 10023, 3, 13697, 9620, 15664, 660, 15681, 4287, 2634, 4417, 9764, 1923, 5219, 9856, 1496, 12230, 18881, 11897, 16415, 551, 18065, 16427, 13195, 10290, 17008, 3927, 4956, 11803, 8526, 177, 10912, 3882, 1520

In [18]:
print(Difference[9][1])

3604


In [42]:
def intersection(A):
    temp = A
    c = []
    for i in range(2):
        print("iteration:",i)
        for j in range(len(temp[i])):
            for k in range(len(temp[i+1])):
                if temp[i][j] == temp[i+1][k]:
                    c.append(temp[i][j])
        temp[i+1] = c
        print(c)
        c = []
        print(c)
        print('----------------------------')
    return temp[i+1]

In [45]:
inter_set = intersection(Difference)
print(inter_set)
print(len(inter_set))
X_new = standardized_X['beijing']
y_new = y['beijing']
X_new = X_new.drop(df.index[inter_set])
y_new = y_new.drop(df.index[inter_set])

iteration: 0
[14705, 17950, 10906, 3814, 19043, 12872, 8863, 11672, 17951, 17666, 3493, 9162, 17650, 2176, 9928, 13222, 6941, 3140, 1905, 1979, 6060, 15850, 8363, 5189, 1405, 5219, 1496, 16415, 551, 18065, 1448, 17401, 17770, 7520, 14855, 6119, 14391, 1533, 5498, 18528, 10431, 10268, 1378, 4441, 14648, 8175, 5472, 15271, 11605, 6171, 13642, 13476, 3272, 290, 15461, 3706, 9034, 3795, 16967, 3058, 6997, 15810, 9858, 10626, 4668, 1713, 12071, 15711, 14338, 15163, 8365, 9244, 6300, 554, 16386, 7866, 412, 3410, 14737, 12274, 4735, 16191, 10340, 982, 14242, 11816, 256, 1447, 10762, 1411, 10602, 14282, 5673, 18605, 18905, 14611, 18750, 8240, 550, 5616, 11271, 10952, 17926, 18248, 7999, 17007, 9345, 3134, 2164, 11906, 9915, 16249, 16795, 73, 12280, 5991, 15254, 18464, 11313, 18988, 10059, 1631, 4666, 11291, 14612, 15932, 1675, 1798, 6607, 14695, 3822, 10914, 1672, 12961, 13036, 15853, 6514, 5084, 134, 18076, 2434, 3420, 8845, 2058, 4572, 6102, 5350, 12884, 3065, 8256, 2038, 11325, 4657, 4891, 

In [46]:
X_trn_bj_d, X_t_bj_d, y_trn_bj_d, y_test_bj_d = train_test_split(X_new, y_new, test_size = 0.3,random_state=0)
model_G = svm.SVC(kernel='rbf', gamma=0.7, C = 1000.0)
model_G.fit(X_trn_bj_d,y_trn_bj_d)
predict_b = model_G.predict(X_t_bj_d)
acc_b = accuracy_score(y_test_bj_d,predict_b)
pred_b = cross_val_predict(model_G, X_t_bj_d, y_test_bj_d, cv=5)
print('beijing')
print("The accuracy is:", acc_b)
print(metrics.classification_report(y_test_bj_d, pred_b))



beijing
The accuracy is: 0.723853049745
             precision    recall  f1-score   support

          1       0.77      0.83      0.80      2316
          2       0.55      0.56      0.55      1546
          3       0.49      0.45      0.47       882
          4       0.43      0.35      0.38       430
          5       0.64      0.57      0.60       515

avg / total       0.63      0.64      0.63      5689



In [ ]:
# credit to the example
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [1]:
models = (svm.LinearSVC(C=10000000.0),
          SVC(kernel='rbf', gamma=1e-07, C = 10000000.0),
          svm.SVC(kernel='rbf', gamma=gamma_best, C = C_best))
models = (clf.fit(X_train, y_train) for clf in models)
# title for the plots
titles = ('SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel')

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = X_train.iloc[:, 0], X_train.iloc[:, 1]
xx, yy = make_meshgrid(X0, X1)

for clf, title, ax in zip(models, titles, sub.flatten()):
    plot_contours(ax, clf, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y_train, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel(X_new.columns[0])
    ax.set_ylabel(X_new.columns[1])
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

plt.show()

NameError: name 'svm' is not defined

In [ ]:
# title for the plots
title = 'beijing'

# Set-up 2x2 grid for plotting.
